In [1]:
import quantecon
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
phi_1, phi_2, phi_3, phi_4, sigma = 0.5, -0.2, 0, 0.5, 0.2

A1 = [[phi_1, phi_2, phi_3, phi_4],
      [1,     0,     0,     0    ],
      [0,     1,     0,     0    ],
      [0,     0,     1,     0    ]]

C1 = [[sigma],[0],[0],[0]]


G1 = [1, 0, 0, 0]

lss = quantecon.LinearStateSpace(A1, C1, G1, mu_0=np.ones(4))

In [6]:
quantecon.LinearStateSpace()

AttributeError: 'LinearStateSpace' object has no attribute 'sigma_0'

In [7]:
lss.simulate(10)

(array([[1.        , 0.40531495, 0.35407884, 0.5684332 , 0.64783237,
         0.47779969, 0.24461409, 0.4314697 , 0.32427137, 0.52854185],
        [1.        , 1.        , 0.40531495, 0.35407884, 0.5684332 ,
         0.64783237, 0.47779969, 0.24461409, 0.4314697 , 0.32427137],
        [1.        , 1.        , 1.        , 0.40531495, 0.35407884,
         0.5684332 , 0.64783237, 0.47779969, 0.24461409, 0.4314697 ],
        [1.        , 1.        , 1.        , 1.        , 0.40531495,
         0.35407884, 0.5684332 , 0.64783237, 0.47779969, 0.24461409]]),
 array([[1.        , 0.40531495, 0.35407884, 0.5684332 , 0.64783237,
         0.47779969, 0.24461409, 0.4314697 , 0.32427137, 0.52854185]]))

In [ ]:
class Person():
    def __init__(self,a,b):
        self.a = a
        self.b = b
    def get_utility(self,mu,sigma):
        return (self.a * mu - 0.5 * self.b * (sigma**2))

class Profession():
    def __init__(self,T,k,income_process):
        self.T = T
        self.k = k
        self.income_process = income_process

class Career():
    def __init__(self,beta,T,k):
        self.A = None
        self.C = None
        self.G = None
        self.mu0 = None
        self.sigma0 = None
        self.beta = beta 
        self.T = None
        self.k = None
        # self.a = None
        # self.b = None

        
    
    def construct_income_process(self,A,C,G,mu0,sigma0):
        self.A = A
        self.C = C
        self.G = G
        self.mu0 = mu0
        self.sigma0 = sigma0
        lss = quantecon.LinearStateSpace(A = A, C= C, G= G,
                                         mu_0=mu0,Sigma_0=sigma0**2)
        return lss
    
    def construct_profession(self,T,k,income_process):
        assert (k <= (T-1)) and (k >=0), 'k must be in [0,1,....T-1]'
        
        self.T = T
        self.k = k

        # state, income_stream = income_process.simulate(T,random_state=random_state)
        # profession = {"T":T,
        #               "k":k,
        #               "income_process":income_process}
        profession = Profession(T,k,income_process)
        return profession
    
    def get_unconditional_mean_PV0(self,profession):
        T = profession['T']
        k = profession['k']
        income_process = profession['income_process']

        m = income_process.moment_sequence()
        discounted_mean = []

        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y = next(m)
            discounted_mean.append(float(μ_y) * (self.beta ** t))
        
        exp_PV0 = sum(discounted_mean[k:])
        return exp_PV0
    
    def get_unconditional_var_PV0(self,profession):
        T = profession['T']
        k = profession['k']
        income_process = profession['income_process']

        m = income_process.moment_sequence()
        discounted_var = []
        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y= next(m)
            discounted_var.append(float(Σ_y) * (self.beta ** (2*t)))
        
        exp_var0 = sum(discounted_var[k:])
        return exp_var0
    
    def get_conditional_mean_PV0(self,profession):
        T = profession.T
        k = profession.K
        income_process = profession.income_process
        income_process.Sigma_0=None  # set x0 to deterministic 

        m = income_process.moment_sequence()
        discounted_mean = []

        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y = next(m)
            discounted_mean.append(float(μ_y) * (self.beta ** (t)))
        
        exp_PV0 = sum(discounted_mean[k:])
        return exp_PV0
    
    def get_conditional_var_PV0(self,profession):
        T = profession['T']
        k = profession['k']
        income_process = profession['income_process']

        m = income_process.moment_sequence()
        income_process.Sigma_0=None  # set x0 to deterministic 

        discounted_var = []
        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y= next(m)
            discounted_var.append(float(Σ_y) * (self.beta ** (2*t)))
        
        exp_var0 = sum(discounted_var[k:])
        return exp_var0

    def construct_person(self,a,b):
        # self.a = a
        # self.b = b

        person = Person(a,b)
        return person
    
    def get_unconditional_EU(self,person,profession):
        mu = self.get_unconditional_mean_PV0(profession)
        var = self.get_unconditional_var_PV0(profession)

        EU = person.a * mu - 0.5 * person.b * var
        return EU
        

    def get_conditional_EU(self,person,profession):
        mu = self.get_conditional_mean_PV0(profession)
        var = self.get_conditional_var_PV0(profession)

        EU = person.a * mu - 0.5 * person.b * var
        return EU
        





        



        